# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [1]:
!pip install -q -U langchain openai

In [24]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [34]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [5]:
!pip install -q -U wikipedia

In [12]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)", 
    load_max_docs=1, # YOUR CODE HERE, 
    doc_content_chars_max=1_000_000### YOUR CODE HERE
    ).load()

barbie_csv_docs = CSVLoader(
    file_path='./barbie_data/barbie.csv',### YOUR CODE HERE, 
    source_column='Review_Url'### YOUR CODE HERE
    ).load()

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =500, ### YOUR CODE HERE,
    chunk_overlap =0, ### YOUR CODE HERE,
    length_function =len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators =["\n==","\n"," "], ### YOUR CODE HERE # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1000, ### YOUR CODE HERE,
    chunk_overlap =50, ### YOUR CODE HERE,
    length_function =len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators =["\n", " "] ### YOUR CODE HERE # keep paragraphs, then sentences
)

chunked_barbie_wikipedia_docs =wikipedia_text_splitter.transform_documents(barbie_wikipedia_docs) ### YOUR CODE HERE
chunked_barbie_csv_docs =csv_text_splitter.transform_documents(barbie_csv_docs) ### YOUR CODE HERE

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [14]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

In [27]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store =LocalFileStore("./shared_cache/")### YOUR CODE HERE

core_embeddings_model =OpenAIEmbeddings()#### YOUR CODE HERE

#core_embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key) ### YOUR CODE HERE

embedder = CacheBackedEmbeddings.from_bytes_store(
core_embeddings_model, store, namespace=core_embeddings_model.model
)

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [28]:
barbie_csv_faiss_retriever =FAISS.from_documents(chunked_barbie_csv_docs,embedder).as_retriever() ### YOUR CODE HERE


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [35]:
# set up BM25 retriever / bag of words
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_barbie_wikipedia_docs
    ### YOUR CODE HERE
)
barbie_wikipedia_bm25_retriever.k = 1 # return 1 document

# set up FAISS vector store
barbie_wikipedia_faiss_store = FAISS.from_documents(
    chunked_barbie_wikipedia_docs,
    embedder
)
barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever],
    weights= [0.25,0.75]### YOUR CODE HERE # should sum to 1
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [39]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    barbie_ensemble_retriever,
    "wikipedia",
    "searches and returns documents regarding the plot, history, and cast of the barbie movie"
)

barbie_csv_retrieval_tool = create_retriever_tool(
    barbie_csv_faiss_retriever,
    "PublicReviews",
    "Searches and returns documents regarding public regiews of the Barbie movie"
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool]

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [45]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(llm, barbie_retriever_tools, verbose=True)

In [46]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `Barbie movie`


[Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, centered in and around the universe of Barbie, Mattel's historic toy doll. Unfortunately, the film tries to be and do much more than what the filmmakers ultimately are able to control and reel in. That a movie about Barbie would be guilty of Americanization is no surprise, and in many ways more than befitting, but the Americanization here isn't just about the history of the toy itself and all its cultural impacts - Barbie also views everything through the eyes of Hollywood anno 2023 - which arguably is a very particular and perhaps not that representative outlook. Instead of utilizing the fine foundation laid in the film's first third, director and co-writer Greta Gervig (Lady Bird, Little Women) gets herself lost in a trite, divisive and most likely futi

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'PublicReviews', 'arguments': '{\n  "__arg1": "Barbie movie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, centered in and around the universe of Barbie, Mattel\'s historic toy doll. Unfortunately, the film tries to be and do much more than what the filmmakers ultimately are able to control and reel in. That a movie about Barbie would be guilty of Americanization is no surprise, and in many ways more than befitting, but the American

In [80]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Barbie (film)`


[Document(page_content='Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu,', metadata={'title': 'Barbie (film)', 'summary': 'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of sel

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [AIMessage(content="I apologize for the confusion. It seems that the information I retrieved is about the 2023 live-action Barbie film directed by Greta Gerwig. I couldn't find a specific list of top-rated Barbie movies. However, there have been numerous computer-animated Barbie films and specials released in the past. Some popular and well-received Barbie movies include:\n\n1. Barbie as Rapunzel (2002)\n2. Barbie in the Nutcracker (2001)\n3. Barbie and the Diamond Castle (2008)\n4. Barbie: Princess Charm School (2011)\n5. Barbie: Fairytopia (2005)\n6. Barbie: Mermaidia (2006)\n7. Barbie: The Princess and the Popstar (2012)\n8. Barbie: Mariposa and the Fairy Princess (2013)\n9. Barbie: Spy Squad (2016)\n10. Barbie: Dolphin Magic (2017)\n\nThese are just a few examples, and there are many more Barbie movies available. The popularity and ratings of these movies may vary depending on personal prefe

In [49]:
barbie_retriever_agent_executor({"input" : "Should i take my husband to see this movie based on a quick summary? My husban likes lego movie."})



> Entering new AgentExecutor chain...
Based on the quick summary of the Barbie movie, it seems to have a similar light-hearted and comedic tone as the Lego Movie. However, since personal preferences can vary, it's difficult to say for certain if your husband would enjoy the Barbie movie. If he enjoyed the Lego Movie and is open to watching other light-hearted and not serious films, there is a chance that he might enjoy the Barbie movie as well. It could be worth giving it a try and seeing if it appeals to his sense of humor and entertainment preferences.

> Finished chain.


{'input': 'Should i take my husband to see this movie based on a quick summary? My husban likes lego movie.',
 'chat_history': [AIMessage(content='Based on the public reviews of the Barbie movie, opinions are mixed. Some people enjoyed the movie and found it entertaining, while others had reservations or found it confusing. \n\nOne review mentioned that the film tries to be more than just a movie about Barbie and explores themes of Americanization and Hollywood culture. The reviewer felt that this approach may not be representative or relatable to everyone, particularly Europeans. \n\nAnother review mentioned that the movie has a fun and colorful atmosphere, with emotional moments and funny comedy. The reviewer appreciated the doll-like aesthetic of the movie, including the plastic food and drink, gliding cars, and doll-like movements of the actors.\n\nOne review described the movie as a darkly satirical look at gender structure in society. It highlighted the comedic premise of the fil

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [81]:


oppenheimer_wikipedia_docs = WikipediaLoader(
    query="Oppenheimer (film)", 
    load_max_docs=1,  
    doc_content_chars_max=1_000_000
    ).load()

oppenheimer_csv_docs = CSVLoader(
    file_path='./oppenheimer_data/oppenheimer.csv',
    source_column='Review_Url'
    ).load()

In [82]:
chunked_opp_wikipedia_docs =wikipedia_text_splitter.transform_documents(oppenheimer_wikipedia_docs) 
chunked_opp_csv_docs = csv_text_splitter.transform_documents(oppenheimer_csv_docs)

In [83]:
# public review Opp retriever
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_csv_docs,embedder).as_retriever() 
# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(chunked_opp_wikipedia_docs)
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(chunked_opp_wikipedia_docs, embedder)
opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever,opp_wikipedia_faiss_retriever], weight=[0.25,0.75]
 ) 

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [86]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])


In [87]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [88]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content="People generally thought highly of the Oppenheimer movie. Critics praised the screenplay, performances of the cast (especially Cillian Murphy and Robert Downey Jr.), and the visuals. It was frequently cited as one of the best films of 2023. On the review aggregator website Rotten Tomatoes, 93% of 451 critics' reviews were positive, with an average rating of 8.6/10. The movie received critical acclaim, with particular praise for the cast performances, screenplay, and visuals. However, some criticism was aimed towards the writing of the female characters. Overall, the movie was considered compelling, engaging, and simple to understand, and it sparked an interest in history.", additional_kwargs={}, example=False)

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [89]:
from langchain.agents import Tool

## barbie_retriever_agent_executor({"input" : "Did people like Barbie"})
## oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

def query_oppenheimer(input): # because we use multi-query, 
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [  ## this is actually 4 soures, but this pattern gives structure, the description helps it to decide which tool to use
    Tool(
        name = "BarbieInfo",#
        func=barbie_retriever_agent_executor.invoke,
        description="useful for when you need to answer questions about Barbie. Input should be a fully formed question."
    ),
    Tool(
        name = "OppenheimerInfo", 
        func=query_oppenheimer, 
        description="useful for when you need to answer questions about Oppenheimer. Input should be a fully formed question."
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [90]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

In [91]:
from langchain import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [92]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain=llm_chain, 
    tools=tools, 
    verbose=True)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=barbenheimer_agent, 
    tools=tools, 
    verbose=True)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [93]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about the Barbie movie.
Action: BarbieInfo
Action Input: What did people like about the Barbie movie?

> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `Barbie (film)`


[Document(page_content=': 97\nReview_Date: 21 July 2023\nAuthor: MrDHWong\nRating: 9\nReview_Title: A darkly satirical look at gender structure in society\nReview: "Barbie" is a comedy film based on the line of fashion dolls by Mattel. Directed and co-written by Greta Gerwig ("Lady Bird", "Little Women") and starring Margot Robbie and Ryan Gosling, it takes every opportunity to revel in its amusing premise with a darkly satirical look at gender structure in society.\nReview_Url: /review/rw9199947/?ref_=tt_urv', metadata={'source': '/review/rw9199947/?ref_=tt_urv', 'row': 97}), Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, cent

{'input': 'What did people like about the Barbie movie?',
 'output': 'People liked the Barbie movie for its satirical approach to gender structure, visual spectacle, creative physical comedy, delightfully absurd opening sequence, theme of empowerment, and overall fun and entertaining nature.'}

In [78]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find out what people liked about the Oppenheimer movie.
Action: OppenheimerInfo
Action Input: "What did people like about the Oppenheimer movie?"
Observation: content="People liked the screenplay, the performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised the accuracy of the film and its ability to make a complicated man's story compelling and engaging. The cinematography was also praised, and viewers enjoyed the three-hour runtime." additional_kwargs={} example=False
Thought:I now know what people liked about the Oppenheimer movie.
Final Answer: People liked the screenplay, the performances of the cast, the visuals, the accuracy, the cinematography, and the three-hour runtime of the Oppenheimer movie.

> Finished chain.


'People liked the screenplay, the performances of the cast, the visuals, the accuracy, the cinematography, and the three-hour runtime of the Oppenheimer movie.'

In [95]:
barbenheimer_agent_chain.run({"input" : "what are the five main themes of Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about the five main themes of the Oppenheimer movie.
Action: OppenheimerInfo
Action Input: What are the five main themes of the Oppenheimer movie?
Observation: content="Based on the information provided, the five main themes of the Oppenheimer movie are:\n\n1. Fear and foreboding about the horror of full-on nuclear conflict\n2. Humanity's capacity for greatness and terribleness\n3. The ethical and moral dilemmas surrounding the creation of the atomic bomb\n4. The influence of the red scare and McCarthyism on J. Robert Oppenheimer's life\n5. The pressure and internal conflict faced by Oppenheimer as he directed the Manhattan Project." additional_kwargs={} example=False
Thought:I now know the final answer
Final Answer: The five main themes of the Oppenheimer movie are fear and foreboding about the horror of full-on nuclear conflict, humanity's capacity for greatness and terribleness, the ethical and moral dilemma

"The five main themes of the Oppenheimer movie are fear and foreboding about the horror of full-on nuclear conflict, humanity's capacity for greatness and terribleness, the ethical and moral dilemmas surrounding the creation of the atomic bomb, the influence of the red scare and McCarthyism on J. Robert Oppenheimer's life, and the pressure and internal conflict faced by Oppenheimer as he directed the Manhattan Project."

In [96]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about both Barbie and Oppenheimer to determine if their movies share similar themes or ideas.
Action: BarbieInfo
Action Input: Did Barbie movies have any specific themes or ideas?

> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Barbie (film series)`


[Document(page_content='Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu,', metadata={'title': 'Barbie (film)', 'summary': 'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a

"Based on the information available, the movies Barbie and Oppenheimer explore different themes and ideas. Barbie movies focus on self-discovery, unrealistic beauty standards, empowerment, and gender structure in society, while Oppenheimer's movie delves into Oppenheimer's internal conflict, the red scare and McCarthyism, and the ethical implications of scientific advancements."

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: